In [1]:
from tqdm import tqdm
from src.data import load_subject, make_kfolds
from src.plots import plot_brain

## train

In [2]:
subject = load_subject('subj05')
config = {'batch_size': 32, 'image_size': 128, 'n_splits': 5}
kfolds = make_kfolds(subject, config)